<a href="https://colab.research.google.com/github/ankur416/Bharatintern/blob/main/Cat_Dog_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dogs-cats-images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F22535%2F28903%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240616%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240616T124614Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D46786e02205b6aa36ca0d085c6e588968bd3e537239d464f46adb6c83f042fa04d8990f52116b8e419c4cffc06afeabd8863a557eb2390d9512168a5b019351e0ae67003b33f76e3d4de33ed6cb76a39da3d5d745c5eb31b5ae30ada7219eca0dcbc3a1e09c4f5a5a5330aff4fe3f276803bc3a7d8faa32b262150f8facf8e4d87780625ba62e74974c6e4e8f8e94bf33ea79c9a7b2eb6e7047f7b838414067bc0c02259838fc45ce500e7195f8d4a440eb57c700eaf5ea100b892107a3bf05cd8cbe85109c7d2e808229de3aee4d73c99d1478bc9f10ee3d66401ba7e78efb33411c228804119f9fe2dbc85197dcddad4f6b42736ed49d2b25c3687bb051061'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 455718914 bytes downloaded
Downloaded and uncompressed: dogs-cats-images
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Streaming output truncated to the last 5000 lines.
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.122.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.1265.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.3605.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.1667.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.1006.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.902.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.1837.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.2154.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.656.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.3431.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.3884.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.2400.jpg
/kaggle/input/dogs-cats-images/dataset/training_set/cats/cat.585.jpg
/kaggle/input/dogs-cats-images/dataset/trai

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
test_d="../input/dogs-cats-images/dog vs cat/dataset/test_set"
train_d="../input/dogs-cats-images/dog vs cat/dataset/training_set"

In [ ]:
train_d_cats= train_d + '/cats'
train_d_dogs=train_d + '/dogs'
test_d_cats= test_d + '/cats'
test_d_dogs= test_d + '/dogs'

In [ ]:
generator= ImageDataGenerator(rescale=1/250, zoom_range=0.25)

In [ ]:
train_data= generator.flow_from_directory(directory= train_d, target_size=(150,150),batch_size=10, class_mode='binary')
test_data= generator.flow_from_directory(directory= test_d, target_size=(150,150), batch_size=10, class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
model= Sequential()
model.add(Conv2D(32,(4,4), input_shape=(train_data.image_shape), activation='ReLU'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
model.add(Conv2D(64,(4,4), activation='ReLU'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
model.add(Conv2D(128,(4,4),activation='ReLU'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.15))
model.add(Flatten())
model.add(Dense(64, activation='ReLU'))
model.add(Dropout(0.15))
model.add(Dense(128, activation='ReLU'))
model.add(Dropout(0.1))
model.add(Dense(len(set(train_data.classes)),activation='Softmax'))
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 147, 147, 32)      1568      
                                                                 
 max_pooling2d (MaxPooling2  (None, 73, 73, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 73, 73, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 70, 70, 64)        32832     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 35, 35, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 35, 35, 64)        0

In [ ]:
model.fit(train_data, steps_per_epoch=700, epochs=10, validation_data=train_data, validation_steps=700)

Epoch 1/10
700/700 [==============================] - 694s 988ms/step - loss: 0.6952 - accuracy: 0.4991 - val_loss: 0.6926 - val_accuracy: 0.5117
Epoch 2/10
700/700 [==============================] - 684s 978ms/step - loss: 0.6928 - accuracy: 0.5271 - val_loss: 0.6845 - val_accuracy: 0.5664
Epoch 3/10
700/700 [==============================] - 681s 973ms/step - loss: 0.6870 - accuracy: 0.5467 - val_loss: 0.6896 - val_accuracy: 0.5351
Epoch 4/10
700/700 [==============================] - 631s 902ms/step - loss: 0.6863 - accuracy: 0.5467 - val_loss: 0.6846 - val_accuracy: 0.5599
Epoch 5/10
700/700 [==============================] - 688s 984ms/step - loss: 0.6846 - accuracy: 0.5547 - val_loss: 0.6782 - val_accuracy: 0.5683
Epoch 6/10
700/700 [==============================] - 683s 976ms/step - loss: 0.6828 - accuracy: 0.5650 - val_loss: 0.6736 - val_accuracy: 0.5700
Epoch 7/10
700/700 [==============================] - 680s 972ms/step - loss: 0.6734 - accuracy: 0.5806 - val_loss: 0.6581 -

In [ ]:
test_loss, test_acc = model.evaluate(test_data, steps=len(test_data))
print('Test Accuracy:', test_acc*100)

200/200 [==============================] - 45s 223ms/step - loss: 0.6340 - accuracy: 0.6590
Test Accuracy: 65.89999794960022


In [ ]:
def test_img(image_directory):
  test_img=image.load_img(image_directory, target_size=(150,150))
  test_img=image.img_to_array(test_img)
  test_img=np.expand_dims(test_img, axis=0)
  result= model.predict(x=test_img)
  print(result)

  if result[0][0] == 1:
    prediction= 'Dog'
  else:
    prediction= 'Cat'
  return prediction

In [ ]:
print(test_img(test_d + '/dogs/dog.4023.jpg'))

1/1 [==============================] - 0s 35ms/step
[[1.2946084e-32 1.0000000e+00]]
Cat
